In [374]:
import numpy as np
import pandas as pd
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

In [375]:
global_temp_country = pd.read_csv('climate_change/GlobalTemperatures.csv')

In [381]:
global_temp_country['year'] = pd.to_datetime(global_temp_country['dt']).dt.year
yearly_data = global_temp_country.groupby('year').agg(
    mean_temp=('LandAverageTemperature', 'mean'),
    mean_temp_uncertainty=('LandAverageTemperatureUncertainty', 'mean')
).reset_index()

yearly_data['upper_bound'] = yearly_data['mean_temp'] + yearly_data['mean_temp_uncertainty']
yearly_data['lower_bound'] = yearly_data['mean_temp'] - yearly_data['mean_temp_uncertainty']

trace0 = go.Scatter(
    x=yearly_data['year'], 
    y=yearly_data['upper_bound'],
    fill=None,
    mode='lines',
    name='Uncertainty top',
    line=dict(color='rgb(0, 255, 255)')
)

trace1 = go.Scatter(
    x=yearly_data['year'], 
    y=yearly_data['lower_bound'],
    fill='tonexty',
    mode='lines',
    name='Uncertainty bot',
    line=dict(color='rgb(0, 255, 255)')
)

trace2 = go.Scatter(
    x=yearly_data['year'], 
    y=yearly_data['mean_temp'],
    name='Average Temperature',
    line=dict(color='rgb(199, 121, 093)')
)

layout = go.Layout(
    xaxis=dict(title='Year'),
    yaxis=dict(title='Average Temperature, °C'),
    title='Average Land Temperature in World',
    showlegend=False
)

fig = go.Figure(data=[trace0, trace1, trace2], layout=layout)

fig.show()